In [1]:
import pandas as pd
import numpy as np
import pickle
import random
import os
import math
import pytz
import sim_functions  as sf
from sklearn.metrics import mean_squared_error as mse
import operator
import time

'''
Context: 
group_id,day_of_week,time_of_day,location,weather,previous_step_count,last_step_count,variation_in_step_counts
'''

A study needs some days. So you can specify days like this: 


In [2]:
data_root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/processed/'


In [3]:
days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=45),freq='30T')

In [4]:
with open('{}merged_est.pkl'.format(data_root),'rb') as f:
    merged  = pickle.load(f)

In [5]:
with open('{}group_lookup_activity.pkl'.format(data_root),'rb') as f:
    gl = pickle.load(f)

In [6]:
#gl

In [7]:
#root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'

In [8]:

#days = pd.date_range(start = merged[1].index[0].tz_localize(None),end =merged[1].index[-1].tz_localize(None),freq='30T')

In [9]:
def adjust_to_thirty_minute(steps):
    return steps.fillna(0).tz_localize(None).resample('30T').sum().fillna(0)

In [10]:
def evaluate_a_person(pid,person_df,k):
    days = pd.date_range(start = person_df.index[0].tz_localize(None),end =person_df.index[-1].tz_localize(None),freq='30T') 
    steps = adjust_to_thirty_minute(person_df['steps'])
    #print(len(days))
    #print(len(steps))
    #return
    all_runs = []
    group_ids = [gl[pid]]
    all_states=[]
    for i in range(k):
        sps,states = sf.simulate_run(1,days,[],group_ids=group_ids)
        #print('real group id')
        #print(group_ids[0])
        all_runs.append(sps)
        all_states.append(states)
    return steps,all_runs,all_states

In [11]:
def get_decision_times(days):
    decision_hours = [10,1,3,5,7]
    decision_times = []
    for d in days:
        new_dt = d
        for h in decision_hours:
            decision_times.append(new_dt.replace(hour=h))
    return decision_times

In [12]:
def evaluate_a_person_simple(pid,person_df,k):
    days = pd.date_range(start = person_df.index[0].tz_localize(None),end =person_df.index[-1].tz_localize(None),freq='30T') 
    steps = adjust_to_thirty_minute(person_df['steps'])
    #print(len(days))
    #print(len(steps))
    #return
    all_runs = []
    group_ids = [gl[pid]]
    #group_ids = [int(random.random()>4.0/36)+1]
    all_states=[]
    dts = get_decision_times(days)
    for i in range(k):
        sps,states = sf.simulate_run(1,days,dts,group_ids=group_ids)
        #print('real group id')
        #print(group_ids[0])
        all_runs.append(np.array(sps[0]).mean())
        all_states.append(states)
    return steps.mean(),all_runs,all_states

In [13]:
def score(gt,sim):
    scores = []
    for s in sim:
        scores.append(mse(gt,np.array(s[0])))
    return np.array(scores)
def score_simple(gt,sim):
    scores = []
    for s in sim:
        scores.append(abs(gt-s))
    return np.array(scores)
def score_all(iters):
    scores = []
    sims ={}
    for k,v in merged.items():
        if k!=4:
            #print(k)
            gt,sim,states = evaluate_a_person_simple(k,v,iters)
            scores.append(score_simple(gt,sim).mean())
            sims[k]=sim
            #print('\n')
    print(np.array(scores).mean())
    return np.array(scores),sims,states
    #scores,sim,states = score_all(10)

In [14]:
#for i in range(10):
start = time.time()
scores,sim,states = score_all(1)
print(np.array(scores).mean())
end = time.time()
print(end-start)

99.9265303797
99.9265303797
227.21320700645447


In [ ]:
104.089741411

In [16]:
weathers = [s[4] for s in states[0]]
set(weathers)
print(weathers.count(0))
print(weathers.count(1))
print(weathers.count(2))
print(weathers.count(3))

623
55
168
1386


In [ ]:
109.668425634
109.668425634

In [15]:
weathers = [s[4] for s in states[0]]
set(weathers)
print(weathers.count(0))
print(weathers.count(1))
print(weathers.count(2))

599
125
1508


In [15]:
weathers = [s[4] for s in states[0]]
set(weathers)
print(weathers.count(0))
print(weathers.count(1))
print(weathers.count(2))

558
60
1614


In [15]:
weathers = [s[4] for s in states[0]]
set(weathers)
print(weathers.count(0))
print(weathers.count(1))
print(weathers.count(2))

77
381
1774


In [18]:
weathers = [s[4] for s in states[0]]
set(weathers)
print(weathers.count(0))
print(weathers.count(1))
print(weathers.count(2))

511
33
1688


In [ ]:
#the location distributions were wrong
#82.7768634577
#gid only - not hwo 215
#tod only - 121.800362209
#dow only - 227.24865704
#tod - dow 122.372640022
#no gid 128.075583847

In [14]:

#for i in range(10):
start = time.time()
scores,sim,states = score_all(1)
print(np.array(scores).mean())
end = time.time()
print(end-start)

113.254912684
113.254912684
350.5237181186676


In [28]:
weathers = [s[7] for s in states[0]]
set(weathers)
#weathers.count(2)
#print(np.array(scores).mean())

{0}

In [19]:
len(states)

1

In [15]:
106.83829381154372
108.48074419049107
##best so far with pre steps
131.39398891766336
##best so far with pre and duration
111.28124570196549

[[[2, 1, 0, 0, 1, 1, 0, 3, 1],
  [2, 1, 0, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 1, 1, 0, 1, 1, 0, 3, 1],
  [2, 0, 1, 0, 1, 1, 0, 3, 1],
  [2, 0, 1, 0, 1, 1, 0, 3, 1],
  [2, 0, 0, 0, 1, 1, 0, 3, 1],
  [2, 0, 0, 0, 1, 1, 0, 3, 1],
  [2, 0, 0, 0, 1, 1, 0, 3, 1],
  [2, 0, 0, 0, 1, 1, 0, 3, 1],
  [2, 0, 0, 1, 1, 1, 2, 3, 1],
  [2, 0,

In [ ]:
185.08084064567225

In [ ]:
103.43882768487441

In [ ]:
34.833737695617799

In [ ]:
140.49494342125112

In [ ]:
35.532149068254718

In [ ]:
34.523242705920609

In [ ]:
34.455529442341401

In [15]:
353.4615149870512
368.71346066177387

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  418.2744974874372,
  418.2744974874372,
  418.2744974874372,
  418.2744974874372,
  418.2744974874372,
  418.2744974

In [16]:
346.899521307648
368.71346066177387

[217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565177758,
 217.69040565

In [14]:
139169.020021**.5

373.0536422835193

In [15]:
136901.28475**.5

370.0017361445754

In [ ]:
#Score: 'gid','tod','dow','loc','yst','pre','var'
92.8514253249
#Score: 'gid','tod','dow','loc','yst','pre','var', adding size constraint
92.8514253249
#Score: no yst
69.0284556908
#Score: no var
61.0556085259
96?
#Score no var and no yst
56.9761605492
56.9887654524
#Score no var and no yst size restrictions
56.6774086091

#Score: gid, tod, dow
34.455529442341401

In [14]:
56.6774086091 (four context)
56.4691856091
58.436632986680479

SyntaxError: invalid syntax (<ipython-input-14-006529efe16f>, line 1)

In [ ]:
56.6774086091 (four context)
56.4691856091
58.436632986680479

In [24]:
def get_alls(gid):
    with open('../../personal_data_functions/data/for_distributions/for_makingn_durt_hwo.pkl','rb') as f:
        current_dists = pickle.load(f)
    alls=[]
    for k,v in current_dists.items():
        if k[0]==str(gid) and len(k)==5:
            alls.extend(v)
    return alls

In [25]:
def evaluate_a_person_baseline(pid,person_df):
    #days = pd.date_range(start = person_df.index[0].tz_localize(None),end =person_df.index[-1].tz_localize(None),freq='30T') 
    steps = adjust_to_thirty_minute(person_df['steps'])
    
    gid = [gl[pid]][0]
    #print(gid)
    
    #mean = (dists['{}-mean'.format(1)][0]+dists['{}-mean'.format(2)][0])/2
    #sc = (dists['{}-mean'.format(1)][1]+dists['{}-mean'.format(2)][1])/2
    alls = get_alls(gid)
    x = np.random.normal(loc=np.array(alls).mean(),scale=np.array(alls).std())
    #mean = dists['{}-mean'.format(gid)][0]
    mn = np.array(alls).mean()
    sc = np.array(alls).std()
    pred = []
    x = np.random.normal(loc=mn,scale=sc)
    while len(pred)<len(steps):
        while x<0:
            x = np.random.normal(loc=mn,scale=sc)
        #pred.append(x)
        pred.append(0)
    #[np.random.normal(loc=mn,scale=sc) for i in range(len(steps))]
    
    
    #pred = [dists['{}-mean'.format(gid)][0] for i in range(len(steps))]
    if len([j for j in pred if j<0])>0:
        print('negs')
  
    #return steps,pred
    return steps.mean(),np.array(pred).mean()

In [26]:

def score_all_baseline():
    scores = []
    for k,v in merged.items():
        if k!=4:
            gt,sim = evaluate_a_person_baseline(k,v)
            #was mse
            scores.append(abs(gt-sim))
            #scores.append(mse(gt,sim))
    return np.array(scores) 

In [27]:
s=score_all_baseline().mean()
s

121.85609299275259

In [21]:
s=score_all_baseline().mean()
s

298.11308723051229

In [19]:
s=score_all_baseline().mean()
s**.5

348.24730361963424

In [23]:
s**.5

352.54246206457952

In [19]:
s

58.436632986680479

In [ ]:
current
325.53227178

In [13]:
325.83
324.956496262
325.721234027
328.080271853
336.136596803
335.347792594

NameError: name 'gt' is not defined

In [ ]:
5
567.813777509

In [21]:
 #s = adjust_to_thirty_minute(merged[1]['steps'])

In [22]:
#idx = days = pd.date_range(start = s.index[0].tz_localize(None),end =s.index[-1].tz_localize(None),freq='30T') 
    
#s.reindex(idx, fill_value=0)

In [8]:
gt,sim,states = evaluate_a_person(3,merged[3],5)

In [11]:
def find_trouble_zones(gt,steps):
    errors = []
    for s in steps:
        these_errors = []
        for i in range(len(s)):
            real = gt[i]
            pred = s[i]
            err = abs(real-pred)
            these_errors.append(err)
        errors.append(these_errors)
    return errors

In [9]:
#errors = find_trouble_zones(gt,sim)

In [12]:
def get_highest_errors(errors,states):
    to_return = {}
    
    for i in range(len(errors)):
        for j in range(len(errors[i][0])):
            state = '-'.join([str(x) for x in states[i][j]])
            #print(state)
            if state not in to_return:
                to_return[state]=[]
            to_return[state].append(errors[i][0][j])
    return {k:np.array(v).mean() for k,v in to_return.items()}
    

In [11]:
#err_states = get_highest_errors(errors,states)

In [20]:
s**.5

352.54246206457952

In [12]:
#errors[0]

In [51]:
st = set([])
for sx in states[0]:
    st.add('-'.join([str(x) for x in sx]))

In [66]:
sorted_x = sorted(err_states.items(), key=operator.itemgetter(1),reverse=True)

In [67]:
sorted_x

[('1-1-1-1-1-3-3-0-1', 1319.0964010000675),
 ('1-0-1-1-1-3-3-0-1', 1188.1783543983031),
 ('1-0-1-1-1-3-1-1-1', 1182.8847732561737),
 ('1-0-2-1-1-3-2-1-1', 1143.4822185831838),
 ('1-0-1-1-1-3-3-1-1', 1141.6783708592334),
 ('1-0-2-1-1-3-3-1-1', 1003.3873050525433),
 ('1-0-1-1-1-3-2-1-1', 997.3688417169833),
 ('1-1-2-1-1-3-3-0-1', 963.71309498290987),
 ('1-0-2-1-1-3-3-0-1', 876.31137714288514),
 ('2-1-1-1-1-3-2-1-1', 808.17743712064248),
 ('1-0-1-1-1-2-1-1-1', 807.3975512239565),
 ('2-1-2-1-1-1-0-1-1', 793.31072399740617),
 ('2-1-2-1-1-0-0-1-1', 789.0143760379982),
 ('2-0-1-1-1-3-2-0-1', 786.10633214133247),
 ('1-0-2-1-1-1-2-1-1', 697.61682063781029),
 ('1-0-1-1-1-1-1-1-1', 649.33916002868625),
 ('1-0-1-1-1-0-3-1-1', 642.31485449686409),
 ('1-0-1-1-1-0-3-0-1', 634.24725522497613),
 ('1-0-2-1-1-1-3-1-1', 630.10660622743421),
 ('1-1-1-1-1-3-1-1-1', 629.7883212274985),
 ('1-0-2-1-1-3-1-1-1', 623.67814272351018),
 ('1-0-2-1-1-0-3-0-1', 604.9002606937554),
 ('1-0-2-1-1-1-3-0-1', 596.0265305703

In [17]:
dists['{}-mean'.format(1)][0] 

217.69040565177758

In [31]:
score(gt,sim).mean()**.5

426.1626217968647

In [28]:
score(gt,sim).mean()**.5

507.58955600646283

You can pass in a bunch of decision times. I just tested with one. 

In [9]:
decision_times

[Timestamp('2015-07-22 10:00:00', freq='30T'),
 Timestamp('2015-07-22 01:00:00', freq='30T'),
 Timestamp('2015-07-22 03:00:00', freq='30T'),
 Timestamp('2015-07-22 05:00:00', freq='30T'),
 Timestamp('2015-07-22 07:00:00', freq='30T'),
 Timestamp('2015-07-22 10:30:00', freq='30T'),
 Timestamp('2015-07-22 01:30:00', freq='30T'),
 Timestamp('2015-07-22 03:30:00', freq='30T'),
 Timestamp('2015-07-22 05:30:00', freq='30T'),
 Timestamp('2015-07-22 07:30:00', freq='30T'),
 Timestamp('2015-07-22 10:00:00', freq='30T'),
 Timestamp('2015-07-22 01:00:00', freq='30T'),
 Timestamp('2015-07-22 03:00:00', freq='30T'),
 Timestamp('2015-07-22 05:00:00', freq='30T'),
 Timestamp('2015-07-22 07:00:00', freq='30T'),
 Timestamp('2015-07-22 10:30:00', freq='30T'),
 Timestamp('2015-07-22 01:30:00', freq='30T'),
 Timestamp('2015-07-22 03:30:00', freq='30T'),
 Timestamp('2015-07-22 05:30:00', freq='30T'),
 Timestamp('2015-07-22 07:30:00', freq='30T'),
 Timestamp('2015-07-22 10:00:00', freq='30T'),
 Timestamp('2

In [4]:
decision_times = pd.DatetimeIndex(['2015-07-25 11:00:00'])
num_people = 2

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [13]:
x=sim_functions.simulate_run(num_people,temp_indices,decision_times)

In [14]:
np.array(x).mean()
#np.array(x).std()

207.01370855122792

In [15]:
np.array(x).mean()
np.array(x).std()

267.8619020654599

In [95]:
all_runs = []
for i in range(5):
    all_runs.append(simulate_run(num_people,days,[]))

In [3]:
root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/temporary/'


In [11]:
    with open('../../personal_data_functions/data/distributions/initial_temperature_distributions_est.pkl','rb') as f:
        loc_lookup = pickle.load(f)

In [12]:
loc_lookup 

{'0-1': [0.990909090909091, 0.00909090909090909],
 '0-10': [0.6503997794320375, 0.3496002205679625],
 '0-11': [0.7888888888888889, 0.2111111111111111],
 '0-12': [0.8661780673181325, 0.13382193268186754],
 '0-2': [0.8857142857142857, 0.11428571428571428],
 '0-7': [0.0, 1.0],
 '0-8': [0.023335985149827633, 0.9766640148501724],
 '0-9': [0.05532015065913371, 0.9446798493408662],
 '1-1': [0.9905498281786942, 0.009450171821305841],
 '1-10': [0.3749491111412675, 0.6250508888587325],
 '1-11': [0.5857007575757576, 0.41429924242424243],
 '1-12': [0.8322751322751323, 0.16772486772486772],
 '1-2': [0.9166666666666666, 0.08333333333333333],
 '1-7': [0.0, 1.0],
 '1-8': [0.010978835978835979, 0.989021164021164],
 '1-9': [0.019736842105263157, 0.9802631578947368]}

In [24]:
np.argmax(np.random.multinomial(100,[0.9166666666666666, 0.08333333333333333]))

0